In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import polars as pl

# Data

In [2]:
import sys
sys.path.append("/home/onyxia/work/data")
from load_data import S3ParquetReader

BUCKET = f"/msimoneau/jane_street_data"
reader = S3ParquetReader(bucket=BUCKET)

FILE_KEY_S3 = "train.parquet/partition_id=0/part-0.parquet"
df_bpe = reader.read_parquet(FILE_KEY_S3)
df_bpe.shape
all_buckets = reader.fs.ls(BUCKET + "/" + "train.parquet/")
all_buckets
# Load all files
data_partitions = []
t0 = time.time()
for i in tqdm(range(10), desc="Partition_id"):
    FILE_KEY_S3 = f"train.parquet/partition_id={i}/part-0.parquet"
    data_partitions.append(reader.read_parquet(FILE_KEY_S3))
data = pl.concat(data_partitions)

delta_t = time.time() - t0
delta_t

Partition_id: 100%|██████████| 10/10 [01:16<00:00,  7.65s/it]


76.53631472587585

In [3]:
data.estimated_size()

16780428789

In [4]:
import polars as pl

threshold = 0.01  # 5 %

null_ratio = (
    data.null_count() / data.height
)

cols_to_keep = [
    c for c in data.columns
    if null_ratio[c].item() <= threshold
]

data_clean = data.select(cols_to_keep).drop_nulls()

In [5]:
print(f"Proportion of lines {data_clean.height/data.height*100}%")

Proportion of lines 97.97116696894699%


In [10]:
data_clean_symb_1 = data_clean.filter(pl.col("symbol_id") == 1)
data_clean_symb_1 = data_clean_symb_1.sort(["date_id", "time_id"])
data_clean_symb_1.write_parquet("/home/onyxia/work/data/data_clean_symb_1.parquet")